In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data Class

In [ ]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #Score of 4 or 5
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        
        

### Load Data

In [ ]:
import json

file_name = '/content/drive/MyDrive/ML_Final_Lab/data/sentiment/Books_small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
        
reviews[5].text
        

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

### Prep Data

In [ ]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)

test_container = ReviewContainer(test)

In [ ]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


#### Bag of words vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# This book is great !
# This book was so bad

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())





This read was a little more intense than the first read, Kamina story is basically told of how she gets screwed up with the wrong guy. From there everything starts getting crazier and I couldn't put the book down! Great job!
[[0. 0. 0. ... 0. 0. 0.]]


#### Linear SVM



















In [ ]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])







array(['NEGATIVE'], dtype='<U8')

# **Hyperparameter Tuning**

In [ ]:
param_grid = {'C': [0.1, 1,10, 20, 1000],
              'gamma': [1, 0.1, 0.01, 0.001],
              'degree':[1,6,25,50,100],
              'kernel': ['linear','rbf','sigmoid','poly']}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
grid = GridSearchCV(SVC(), param_grid, refit = False, verbose = 1)
grid.fit(train_x_vectors,train_y)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 20, 1000],
                         'degree': [1, 6, 25, 50, 100],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear', 'rbf', 'sigmoid', 'poly']},
             refit=False, verbose=1)

In [ ]:
grid.best_params_

{'C': 10, 'degree': 1, 'gamma': 1, 'kernel': 'rbf'}

In [ ]:
svm_model = svm.SVC(C=10, degree= 1, gamma= 1, kernel= 'rbf')
svm_model.fit(train_x_vectors, train_y)
svm_model.score(test_x_vectors, test_y)

0.8197115384615384

## Evaluation

In [ ]:
# Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(svm_model.score(test_x_vectors, test_y))


0.8076923076923077
0.8197115384615384


**Prediction**

In [ ]:
test_set = ['waste of money', "bad book do not buy", 'horrible waste of time', 'good product', 'bad product']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)



array(['NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE'],
      dtype='<U8')

In [ ]:
svm_model.predict(new_test)

array(['NEGATIVE', 'NEGATIVE', 'NEGATIVE', 'POSITIVE', 'NEGATIVE'],
      dtype='<U8')